# Data Preparation and Processing

In [27]:
import xarray as xr
import numpy as np

In [2]:
era5 = xr.open_zarr(
    'gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_with_poles_conservative.zarr',
    chunks={'time': 48},
    consolidated=True,
    )
print("Model surface dataset size {} GiB".format(era5.nbytes/(1024**3)))
era5

Model surface dataset size 75.83941642194986 GiB


/usr/local/anaconda3/envs/mlg_310/lib/python3.10/site-packages/xarray/core/dataset.py:273: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 48. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


<xarray.Dataset> Size: 81GB
Dimensions:                                           (time: 92040,
                                                       longitude: 64,
                                                       latitude: 32, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 256B ...
  * level                                             (level) int64 104B 50 ....
  * longitude                                         (longitude) float64 512B ...
  * time                                              (time) datetime64[ns] 736kB ...
Data variables: (12/38)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 754MB dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 754MB dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    10m_wind_speed                                    (time, longitude, latitude) float32 754MB dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    2m_temperature                                    (time, longitude, latitude) float32 754MB dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    ...                                                ...
    type_of_high_vegetation                           (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    type_of_low_vegetation                            (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    u_component_of_wind                               (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(48, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind                               (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(48, 13, 64, 32), meta=np.ndarray>
    vertical_velocity                                 (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(48, 13, 64, 32), meta=np.ndarray>
    wind_speed                                        (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(48, 13, 64, 32), meta=np.ndarray>

### The dataset is still huge, we shrink it

In [28]:
# Select specific data variables
selected_vars = [
    # '10m_u_component_of_wind', '10m_v_component_of_wind', 
    '10m_wind_speed', 
    '2m_temperature', 
    # 'mean_sea_level_pressure', 'sea_surface_temperature', 
    # 'surface_pressure', 'toa_incident_solar_radiation_6hr', 
    # 'total_cloud_cover', 
    'total_precipitation_6hr']
era5_selected = era5[selected_vars]
print("Model surface dataset size {} GiB".format(era5_selected.nbytes/(1024**3)))
era5_selected

Model surface dataset size 2.107314884662628 GiB


<xarray.Dataset> Size: 2GB
Dimensions:                  (time: 92040, longitude: 64, latitude: 32)
Coordinates:
  * latitude                 (latitude) float64 256B -90.0 -84.19 ... 84.19 90.0
  * longitude                (longitude) float64 512B 0.0 5.625 ... 348.8 354.4
  * time                     (time) datetime64[ns] 736kB 1959-01-02 ... 2021-...
Data variables:
    10m_wind_speed           (time, longitude, latitude) float32 754MB dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    2m_temperature           (time, longitude, latitude) float32 754MB dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    total_precipitation_6hr  (time, longitude, latitude) float32 754MB dask.array<chunksize=(48, 64, 32), meta=np.ndarray>

In [34]:
# Select a specific range of dimensions
era5_reduced = era5_selected.sel(
    # lat=slice(-90, 90),  # Select latitude range
    # lon=slice(-180, 180),  # Select longitude range
    time=slice('2020-01-01', '2021-01-01')  # Select time range
)
print("Model surface dataset size {} MiB".format(era5_reduced.nbytes/(1024**2)))
era5_reduced

Model surface dataset size 34.418182373046875 MiB


<xarray.Dataset> Size: 36MB
Dimensions:                  (time: 1468, longitude: 64, latitude: 32)
Coordinates:
  * latitude                 (latitude) float64 256B -90.0 -84.19 ... 84.19 90.0
  * longitude                (longitude) float64 512B 0.0 5.625 ... 348.8 354.4
  * time                     (time) datetime64[ns] 12kB 2020-01-01 ... 2021-0...
Data variables:
    10m_wind_speed           (time, longitude, latitude) float32 12MB dask.array<chunksize=(20, 64, 32), meta=np.ndarray>
    2m_temperature           (time, longitude, latitude) float32 12MB dask.array<chunksize=(20, 64, 32), meta=np.ndarray>
    total_precipitation_6hr  (time, longitude, latitude) float32 12MB dask.array<chunksize=(20, 64, 32), meta=np.ndarray>

## Final Dataset size: 34 MiB

In [35]:
has_nans = era5_reduced.isnull().any()
has_nans

<xarray.Dataset> Size: 3B
Dimensions:                  ()
Data variables:
    10m_wind_speed           bool 1B dask.array<chunksize=(), meta=np.ndarray>
    2m_temperature           bool 1B dask.array<chunksize=(), meta=np.ndarray>
    total_precipitation_6hr  bool 1B dask.array<chunksize=(), meta=np.ndarray>

In [36]:
# in case we have nan values, but right now we don't

def fill_na_with_neighbors(da):
    """
    Fill NaN values in the DataArray by averaging their neighbors.
    """
    isnan = np.isnan(da)
    avg_neighbors = np.zeros_like(da)
    count_neighbors = np.zeros_like(da)

    # Shift in all directions and sum up neighbors
    for shift in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        shifted = np.roll(da, shift=shift, axis=(0, 1))
        mask_shifted = np.roll(isnan, shift=shift, axis=(0, 1))
        avg_neighbors += np.where(mask_shifted, 0, shifted)
        count_neighbors += ~mask_shifted

    avg_neighbors = np.where(count_neighbors > 0, avg_neighbors / count_neighbors, np.nan)
    filled = np.where(isnan, avg_neighbors, da)
    return filled

# # Apply the function to all DataArrays in the dataset
# ds_filled = ds.apply(lambda da: xr.apply_ufunc(fill_na_with_neighbors, da))

# # Save the filled dataset
# ds_filled.to_netcdf('filled_dataset.nc')

In [37]:
# Save the reduced dataset to a new file
era5_reduced.to_netcdf('era5.nc')